In [9]:
pip install torch torchvision torchaudio

In [10]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, confusion_matrix,
                             ConfusionMatrixDisplay, RocCurveDisplay)
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

In [11]:
# Load dataset
df = pd.read_csv('Heart-Disease-Dataset.csv')
df = df.dropna()
df['target'] = df['target'].apply(lambda x: 1 if x > 0 else 0)

In [12]:
# Split features and target
X = df.drop('target', axis=1).values
y = df['target'].values

In [13]:
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [14]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [15]:
# Reshape for CNN-LSTM: (samples, time_steps=1, features)
# X_train = X_train.reshape(-1, 1, X.shape[1]).astype(np.float32)
# X_test = X_test.reshape(-1, 1, X.shape[1]).astype(np.float32)

In [53]:
# No reshape — keep raw shape (N, 13)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # (N, 1, 13)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)    # (N, 1, 13)

y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)


In [54]:
# Dataset & Dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [55]:
class CNNLSTM(nn.Module):
    def __init__(self, input_features):
        super(CNNLSTM, self).__init__()
        
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.dropout = nn.Dropout(0.3)
        
        # Adjusted feature length after Conv + Pool
        conv_output_size = (input_features - 2) // 2  # kernel_size=3, then pool=2
        self.lstm = nn.LSTM(input_size=64, hidden_size=100, batch_first=True)
        
        self.fc1 = nn.Linear(100, 50)
        self.fc2 = nn.Linear(50, 1)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # shape: (batch, 64, conv_out)
        x = self.dropout(x)
        x = x.permute(0, 2, 1)  # shape: (batch, seq_len, features) for LSTM
        x, _ = self.lstm(x)
        x = x[:, -1, :]  # take the last output from LSTM
        x = self.dropout(torch.relu(self.fc1(x)))
        x = torch.sigmoid(self.fc2(x))
        return x


In [56]:
# Instantiate model
model = CNNLSTM(13)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [57]:
# Train the model
epochs = 100
train_losses = []

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    train_losses.append(epoch_loss / len(train_loader))
    print(f"Epoch {epoch+1}/{epochs}, Loss: {train_losses[-1]:.4f}")


RuntimeError: Expected 2D (unbatched) or 3D (batched) input to conv1d, but got input of size: [32, 1, 1, 13]

In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).numpy()
    y_pred_class = (y_pred > 0.5).astype(int)

In [ ]:
# Metrics
accuracy = accuracy_score(y_test, y_pred_class)
precision = precision_score(y_test, y_pred_class)
recall = recall_score(y_test, y_pred_class)
f1 = f1_score(y_test, y_pred_class)
auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"AUC: {auc:.4f}")

In [ ]:
# Plot loss
plt.plot(train_losses)
plt.title("Training Loss over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_class)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title("Confusion Matrix")
plt.show()


In [ ]:
# ROC Curve
RocCurveDisplay.from_predictions(y_test, y_pred)
plt.title("ROC Curve")
plt.show()